# How to use it

In [1]:
class MyClass(object):

    @property
    def my_property(self):
        # In reality, this might represent a database call or time
        # intensive task like calling a third-party API.
        print('Computing my_property...')  
        return 42

In [2]:
my_object = MyClass()
my_object.my_property

Computing my_property...


42

In [3]:
my_object.my_property

Computing my_property...


42

In [4]:
from cached_property import cached_property

class MyClass(object):

    @cached_property
    def my_cached_property(self):
        print('Computing my_cached_property...')
        return 42

In [5]:
my_object = MyClass()
my_object.my_cached_property

Computing my_cached_property...


42

In [6]:
my_object.my_cached_property

42

# Inspecting the cache

In [7]:
from cached_property import cached_property, cached_properties, is_cached

In [8]:
class MyClass(object):

    @cached_property
    def my_cached_property(self):
        print('Computing my_cached_property...')
        return 42
    
    @cached_property
    def my_second_cached_property(self):
        print('Computing my_second_cached_property...')
        return 51

In [9]:
my_object = MyClass()

In [10]:
for property_name in cached_properties(my_object):
    print(property_name)

my_cached_property
my_second_cached_property


In [11]:
my_object.my_cached_property

Computing my_cached_property...


42

In [12]:
is_cached(my_object, 'my_cached_property')

True

In [13]:
is_cached(my_object, 'my_second_cached_property')

False

# Invalidating the cache

In [14]:
from cached_property import cached_property, un_cache, delete_cache

In [15]:
class MyClass(object):

    @cached_property
    def my_cached_property(self):
        print('Computing my_cached_property...')
        return 42
    
    @cached_property
    def my_second_cached_property(self):
        print('Computing my_second_cached_property...')
        return 51

In [16]:
my_object = MyClass()

In [17]:
my_object.my_cached_property

Computing my_cached_property...


42

In [18]:
my_object.my_second_cached_property

Computing my_second_cached_property...


51

In [19]:
un_cache(my_object, 'my_cached_property')

In [20]:
my_object.my_cached_property

Computing my_cached_property...


42

In [21]:
my_object.my_second_cached_property

51

In [22]:
delete_cache(my_object)

In [23]:
my_object.my_cached_property

Computing my_cached_property...


42

In [24]:
my_object.my_second_cached_property

Computing my_second_cached_property...


51

# Property deleting the cache

In [25]:
from cached_property import cached_property, property_deleting_cache

In [26]:
class MyClass(object):

    def __init__(self, my_parameter):
        self.my_parameter = my_parameter
    
    @property_deleting_cache
    def my_parameter(self):
        "A parameter that deletes the cache when set or deleted."
        print('Accessing my_parameter...')
        
    @cached_property
    def my_cached_property(self):
        print('Computing my_cached_property...')
        return self.my_parameter + 1

In [27]:
my_object = MyClass(my_parameter=41)
my_object.my_cached_property

Computing my_cached_property...
Accessing my_parameter...


42

In [28]:
my_object.my_cached_property

42

In [29]:
my_object.my_parameter = 50
my_object.my_cached_property

Computing my_cached_property...
Accessing my_parameter...


51

In [30]:
my_object.my_cached_property

51

# Working with Threads

In [31]:
from cached_property import threaded_cached_property

class MyClass(object):

    @threaded_cached_property
    def my_cached_property(self):
        print('Computing my_cached_property...')
        return 42

In [32]:
from threading import Thread
my_object = MyClass()
threads = []
for x in range(10):
    thread = Thread(target=lambda: my_object.my_cached_property)
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

Computing my_cached_property...


# Working with async/await (Python 3.5+)

In [33]:
# This is just a trick to make asyncio work in jupyter.
# Cf. https://markhneedham.com/blog/2019/05/10/jupyter-runtimeerror-this-event-loop-is-already-running/
import nest_asyncio
nest_asyncio.apply()

In [34]:
from cached_property import cached_property

class MyClass(object):

    @cached_property
    async def my_cached_property(self):
        print('Computing my_cached_property...')
        return 42

In [35]:
async def print_my_cached_property():
    my_object = MyClass()
    print(await my_object.my_cached_property)
    print(await my_object.my_cached_property)
    print(await my_object.my_cached_property)
import asyncio
asyncio.get_event_loop().run_until_complete(print_my_cached_property())

Computing my_cached_property...
42
42
42


# Timing out the cache

In [36]:
import random
from cached_property import cached_property_with_ttl

class MyClass(object):

    @cached_property_with_ttl(ttl=2) # cache invalidates after 2 seconds
    def my_cached_property(self):
        return random.randint(1, 100)

In [37]:
my_object = MyClass()

In [38]:
my_object.my_cached_property

66

In [39]:
my_object.my_cached_property

66

In [40]:
from time import sleep
sleep(3)  # Sleeps long enough to expire the cache

In [41]:
my_object.my_cached_property

45